In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda, Dropout, BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
import pandas as pd
from PIL import Image
import urllib
import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time

: 

In [4]:
train_dataset = pd.read_csv('C:/Users/JP/Desktop/final signature verification/sign_data/train_data.csv')
test_dataset = pd.read_csv('C:/Users/JP/Desktop/final signature verification/sign_data/test_data.csv')

In [5]:
train_dir = "C:/Users/J/train"
test_dir = "C:/Users/JP/Desktop/final signature verification/sign_data/test"

In [6]:
class DataLoader:
  #constructor
  def __init__(self, dataset, batch_size,dir):
    self.dataset = dataset
    self.batch_size = batch_size
    self.dir = dir
  #shuffler
  def shuffle(self):
    return self.dataset.sample(frac=1)
  #generator
  def datagen(self):
    num_samples = len(self.dataset)
    while True:
        # shuffling the samples
        self.dataset = self.shuffle()
        for batch in range(1, num_samples, self.batch_size):
            image1_batch_samples = self.dir + "/" + self.dataset.iloc[:, 0][batch:batch + self.batch_size]
            image2_batch_samples = self.dir + "/" + self.dataset.iloc[:, 1][batch:batch + self.batch_size]
            label_batch_samples = self.dataset.iloc[:, 2][batch:batch + self.batch_size]
            Image1, Image2, Label = [], [], []
            for image1, image2, label in zip(image1_batch_samples, image2_batch_samples, label_batch_samples):
                # append them to Images directly
                image1_data = Image.open(image1)
                image2_data = Image.open(image2)
                # resizing the images
                image1_data = image1_data.resize((112, 112))
                image2_data = image2_data.resize((112, 112))
                # converting to array
                image1_data = img_to_array(image1_data)
                image2_data = img_to_array(image2_data)
                Image1.append(image1_data)
                Image2.append(image2_data)
                Label.append(label)
            # convert each list to numpy arrays to ensure that they get processed by fit function
            Image1 = np.asarray(Image1).astype(np.float32)
            Image2 = np.asarray(Image2).astype(np.float32)
            Label = np.asarray(Label).astype(np.float32)
            yield [Image1, Image2], Label


In [7]:
train_set, val_set = train_test_split(train_dataset, test_size=0.25)


In [8]:
train_gen= DataLoader(train_set,1024,train_dir)
val_gen = DataLoader(val_set,1024,train_dir)

In [9]:
train_batch = next(train_gen.datagen())
print("Train batch images shape:", train_batch[0][0].shape, train_batch[0][1].shape)
print("Train batch labels shape:", train_batch[1].shape)


Train batch images shape: (1024, 112, 112, 3) (1024, 112, 112, 3)
Train batch labels shape: (1024,)


In [10]:
def custom_siamese_model(input_shape):
    input1 = Input(input_shape)
    input2 = Input(input_shape)
    
    model = Sequential()
    model.add(Conv2D(16, (3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))

    # Call the model with the inputs
    embedding1 = model(input1)
    embedding2 = model(input2)

    loss_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    manhattan_distance = loss_layer([embedding1, embedding2])
    output = Dense(1, activation='sigmoid')(manhattan_distance)
    network = Model(inputs=[input1, input2], outputs=output)
    return network

In [11]:
early_stopper =  EarlyStopping(monitor='val_loss',min_delta=0,patience=3,verbose=1)
custom_callback = [early_stopper]

In [12]:
model = custom_siamese_model((112,112,3))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 128)          1802016     ['input_1[0][0]',                
                                                                  'input_2[0][0]']            

In [13]:
optimizer = Adam(lr = 1e-4)
model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=['accuracy'])


In [14]:
print("Initializing Training !!")
history = model.fit_generator(
    train_gen.datagen(),
    verbose=1,
    steps_per_epoch=100,  # set appropriate steps_per_epoch
    epochs=30,
    validation_data=val_gen.datagen(),
    validation_steps=100,  # set appropriate validation_steps
    callbacks=custom_callback
)


Initializing Training !!


C:\Users\JP\AppData\Local\Temp\ipykernel_7564\3002214753.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
100/100 [==============================] - 1259s 13s/step - loss: 2.1577 - accuracy: 0.5807 - val_loss: 0.4354 - val_accuracy: 0.7196
Epoch 2/30
100/100 [==============================] - 1258s 13s/step - loss: 0.2337 - accuracy: 0.9145 - val_loss: 0.3760 - val_accuracy: 0.8276
Epoch 3/30
100/100 [==============================] - 1249s 13s/step - loss: 0.0751 - accuracy: 0.9788 - val_loss: 0.4006 - val_accuracy: 0.7926
Epoch 4/30
100/100 [==============================] - 1261s 13s/step - loss: 0.0281 - accuracy: 0.9928 - val_loss: 0.2006 - val_accuracy: 0.9262
Epoch 5/30
100/100 [==============================] - 1266s 13s/step - loss: 0.0157 - accuracy: 0.9957 - val_loss: 0.0470 - val_accuracy: 0.9979
Epoch 6/30
100/100 [==============================] - 1271s 13s/step - loss: 0.0074 - accuracy: 0.9979 - val_loss: 0.0367 - val_accuracy: 0.9958
Epoch 7/30
100/100 [==============================] - 1253s 13s/step - loss: 0.0077 - accuracy: 0.9978 - val_loss: 0.0134 - val_ac

In [15]:
model.save('sign.h5')

In [2]:
# from tensorflow.keras.models import load_model
# from tensorflow.keras.preprocessing import image

In [18]:
# model = load_model('sign.h5')
